In [84]:
#change the path to your absolute path, but only to the point of Localenvpc11
#all the rest should be exactly the same because I am using relative paths from this point
%cd /Users/kroman/Documents/SOHAR/Localenvpc11

/Users/kroman/Documents/SOHAR/Localenvpc11


In [85]:
#commented this out for now because it takes a lot of time and is not strictly necessary at this point
# from roboflow import Roboflow
# rf = Roboflow(api_key="1gq5kPILtMw4wF64rMAM")
# project = rf.workspace("oman-sipc").project("fenderdefectdataweek7label")
# dataset = project.version(1).download("yolov8")

In [86]:
#added this snippet so that the predicition code can also be run with just CPU (so on the work laptop 
#in the future without errors appearing)
import torch
from torch.backends import cudnn

# Check if GPU is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device('cuda:0')
    print("CUDA Available: Yes")
    print("Using GPU.")
    print("cuDNN Available: ", cudnn.is_available())
    print("cuDNN version: ", cudnn.version())
    print("Current CUDA Device: ", torch.cuda.current_device())
    print("CUDA Device Count: ", torch.cuda.device_count())
    print("CUDA Device Name: ", torch.cuda.get_device_name(0))
else:
    # Set the device to CPU
    device = torch.device('cpu')
    print("CUDA Available: No")
    print("Using CPU.")

# Your code can now use the 'device' variable to place tensors and models on the chosen device.
# For example:
# tensor_on_device = your_tensor.to(device)
# model_on_device = your_model.to(device)


CUDA Available: No
Using CPU.


In [87]:
from ultralytics import YOLO
import os

# Load a pretrained YOLOv8n-cls Classify model
# model_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/ultralytics-main/runs/train_old/best.pt"
# modelbest = YOLO(model_path)
#uncomment the two lines above and comment out two lines below
model_path = "Localyolo/ultralytics-main/runs/train_old/best.pt"
modelbest = YOLO(model_path)

# Run inference on a video
# video_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/TestVideos/Blanco/Side2.mp4"
# result_path = "C:/Users/s2018293031/Toon/Localenv/Localyolo/TestVideos/Predicted"
video_path = "Localyolo/TestVideos/Blanco/Side2.mp4"

result_path = "Localyolo/TestVideos/Predicted"

# Check if the results_path directory exists
if not os.path.exists(result_path):
    # Create the directory if it doesn't exist
    os.makedirs(result_path)
    print(f"Directory '{result_path}' created.")
else:
    print(f"Directory '{result_path}' already exists.")


Directory 'Localyolo/TestVideos/Predicted' already exists.


In [76]:
results = modelbest.predict(video_path, save=True, save_crop=True, project=result_path,
                            max_det=5)  # results list for each frame , save_crop=True, save_txt=True
#print(results)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/315) /Users/kroman/Documents/SOHAR/Localenvpc11/Localyolo/TestVideos/Blanco/Side2.mp4: 384x640 1 Missing plate, 316.2ms
video 1/1 (2/315) /Users/kroman/Documents/SOHAR/Localenvpc11/Localyolo/TestVideos/Blanco/Side2.mp4: 384x640 1 Missing plate, 381.9ms
video 1/1 (3/315) /Users/kroman/Documents/SOHAR/Localenvpc11/Localyolo/TestVideos/Blanco/Side2.mp4: 384x640 1 Missing plate, 430.2ms
video 1/1 (4/315) /Users/kroman/Documents/SOHAR/Localenvp

In [77]:
print(results)

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'Loose plate', 1: 'Missing plate', 2: 'Rubber crack'}
orig_img: array([[[175, 173, 167],
        [175, 173, 167],
        [175, 173, 167],
        ...,
        [119,  91,  54],
        [126,  98,  61],
        [130, 102,  65]],

       [[175, 173, 167],
        [175, 173, 167],
        [175, 173, 167],
        ...,
        [111,  83,  46],
        [112,  84,  47],
        [115,  87,  49]],

       [[176, 174, 168],
        [176, 174, 168],
        [176, 174, 168],
        ...,
        [109,  81,  44],
        [103,  75,  38],
        [101,  73,  35]],

       ...,

       [[221, 221, 221],
        [221, 221, 221],
        [221, 221, 221],
        ...,
        [ 33,  24,   5],
        [ 33,  24,   5],
        [ 33,  24,   5]],

       [[221, 221, 221],
        [221, 221, 221],
        [221, 221, 221],
        ...,
        [ 27,  23,  10],
   

In [78]:
import pickle

# Assuming results_object is the variable containing your Results object
with open('results.pkl', 'wb') as f:
    pickle.dump(results, f)

In [79]:
from datetime import timedelta
import cv2
import pandas as pd
from config import *

#this can be used later if we want to centralize video_paths and best_model from dictionaries stored in config.py
# def process_results(video_key, results):
#     video_path = video_paths.get(video_key)
#     if video_path is None:
#         raise ValueError(f"Video key '{video_key}' not found in config.py")

def process_results(video_path, results):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)

    detection_data = []

    for frame_idx, result in enumerate(results):
        frame_time = frame_idx / fps

        boxes = result.boxes.xyxy

        if result.boxes.conf.numel() > 0:
            confidences = result.boxes.conf.tolist()
        else:
            confidences = None

        if result.boxes.cls.numel() > 0:
            classes = result.boxes.cls.tolist()
        else:
            classes = None

        if confidences is not None and classes is not None and boxes is not None and len(boxes) > 0:
            for box, conf, class_id in zip(boxes, confidences, classes):
                x_min, y_min, x_max, y_max = map(float, box[:4])
                class_name = modelbest.names[int(class_id)]

                detection_data.append({
                    "Timestamp": frame_time,
                    "Class": class_name,
                    "Confidence": conf,
                    "Bounding Box": (x_min, y_min, x_max, y_max)
                })

    df = pd.DataFrame(detection_data)

    df[["Timestamp", "Confidence"]] = df[["Timestamp", "Confidence"]].round(3)
    df["Bounding Box"] = df["Bounding Box"].apply(lambda box: tuple(round(coord, 3) for coord in box))

    df['Frame Number'] = (df['Timestamp'] * fps).astype(int)

    df.insert(0, "Fender ID", 1)
    df.insert(1, "Defect ID", 1)

    defect_id = 1
    last_timestamp = df.iloc[0]["Timestamp"]
    last_class = df.iloc[0]["Class"]

    for index, row in df.iterrows():
        time_difference = row["Timestamp"] - last_timestamp
        same_class = row["Class"] == last_class

        if time_difference > 0.15 or not same_class:
            defect_id += 1
        last_timestamp = row["Timestamp"]
        last_class = row["Class"]

        df.at[index, "Defect ID"] = defect_id

    cap.release()

    return df

In [80]:
df.head(30)

,Fender ID,Defect ID,Timestamp,Class,Confidence,Bounding Box,Frame Number
0,1,1,0.000,Missing plate,0.331,"(970.642, 153.215, 1009.886, 261.395)",0
1,1,1,0.033,Missing plate,0.307,"(971.419, 155.801, 1008.935, 264.596)",0
2,1,1,0.067,Missing plate,0.303,"(970.706, 156.468, 1009.944, 267.414)",2
3,1,1,0.100,Missing plate,0.329,"(971.409, 160.8, 1010.206, 269.667)",3
4,1,1,0.133,Missing plate,0.323,"(972.087, 163.185, 1011.535, 273.321)",3
5,1,1,0.200,Missing plate,0.289,"(971.533, 166.892, 1014.15, 278.102)",6
6,1,1,0.233,Missing plate,0.324,"(971.884, 167.956, 1016.167, 282.349)",6
7,1,1,0.267,Missing plate,0.347,"(972.586, 171.129, 1016.003, 285.79)",8
8,1,1,0.300,Missing plate,0.369,"(973.184, 171.814, 1015.942, 288.116)",9
9,1,1,0.333,Missing plate,0.376,"(973.695, 174.399, 1016.95, 290.747)",9


In [81]:
print(results[0].save_dir)

Localyolo/TestVideos/Predicted/predict10


In [82]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import pandas as pd
from PyPDF2 import PdfMerger


# Set up PDF file
path_topdf = results[0].save_dir
pdf_filename = f"{path_topdf}/detection_report.pdf"
pdf_pages = PdfPages(pdf_filename)

# Create a figure and axes for the table
fig, ax = plt.subplots(figsize=(8.5, 11))
ax.axis('off')  # Turn off axis labels and ticks

# Create a table from the DataFrame and add it to the axes
table = ax.table(cellText=df.values, colLabels=df.columns, cellLoc='center', loc='center')

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.2)  # Increase table size for better readability

# Adjust column widths based on content
table.auto_set_column_width([i for i in range(len(df.columns))])

# Save the current figure to the PDF file
pdf_pages.savefig(fig, bbox_inches='tight')
plt.close()

# Close the PDF file
pdf_pages.close()

# Optionally, you can add a title page to the PDF using reportlab
title_page_filename = f"{path_topdf}/title_page.pdf"
canvas_obj = canvas.Canvas(title_page_filename, pagesize=letter)
canvas_obj.setFont("Helvetica", 14)
canvas_obj.drawCentredString(300, 700, "Detection Report")
canvas_obj.save()

# Create a PdfMerger object
merger = PdfMerger()

# Add the title page and dataframe to the merged PDF
merger.append(title_page_filename)
merger.append(pdf_filename)

# Output the final merged PDF
merged_pdf_filename = f"{path_topdf}/final_report.pdf"
merger.write(merged_pdf_filename)
merger.close()


In [57]:
print(results[0].boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.])
conf: tensor([0.3307])
data: tensor([[9.7064e+02, 1.5321e+02, 1.0099e+03, 2.6139e+02, 3.3068e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[990.2639, 207.3046,  39.2446, 108.1799]])
xywhn: tensor([[0.5158, 0.1919, 0.0204, 0.1002]])
xyxy: tensor([[ 970.6415,  153.2146, 1009.8862,  261.3945]])
xyxyn: tensor([[0.5055, 0.1419, 0.5260, 0.2420]])
